# Test Camera Setup for Autonomous Racing

This notebook sets up the camera system for data collection and autonomous driving.

## What this does:
- Tests camera functionality
- Captures training images
- Images may apear yellow but it should be fine while training
- 
## Requirements:
- CSI camera connected, make sure there are no lose connections and make sure the cable is connected in the correct orientation
- GStreamer working - the way we will access our cam(gstreamer not working is usually due to perms issue in nvargus daemon, make sure all background processes are terminated)
- OpenCV installed - (for formatting)

In [1]:
import cv2
import time
import numpy as np
import subprocess
import os

class AutonomousRacecarCamera:
    def __init__(self, mode='inference', width=640, height=480, fps=21):
        self.mode = mode
        self.width = width
        self.height = height
        self.fps = fps
        
        if mode in ['inference', 'training']:
            self.target_size = (224, 224)
        else:
            self.target_size = (width, height)
            
        self.running = False
        self.last_image = None
        
    def start(self):
        if self._test_camera():
            self.running = True
            return True
        return False
    
    def _test_camera(self):
        cmd = [
            'gst-launch-1.0',
            'nvarguscamerasrc', 'num-buffers=1',
            '!', f'video/x-raw(memory:NVMM),width={self.width},height={self.height}',
            '!', 'nvvidconv',
            '!', 'jpegenc',
            '!', 'filesink', 'location=/tmp/test_camera.jpg'
        ]
        
        try:
            result = subprocess.run(cmd, capture_output=True, timeout=10)
            return result.returncode == 0 and os.path.exists('/tmp/test_camera.jpg')
        except:
            return False
    
    def read(self):
        if not self.running:
            return self.last_image
        
        temp_file = f"/tmp/camera_frame_{int(time.time() * 1000)}.jpg"
        
        cmd = [
            'gst-launch-1.0',
            'nvarguscamerasrc', 'num-buffers=1',
            '!', f'video/x-raw(memory:NVMM),width={self.width},height={self.height}',
            '!', 'nvvidconv',
            '!', 'jpegenc',
            '!', 'filesink', f'location={temp_file}'
        ]
        
        try:
            result = subprocess.run(cmd, capture_output=True, timeout=5)
            
            if result.returncode == 0 and os.path.exists(temp_file):
                img = cv2.imread(temp_file)
                os.remove(temp_file)
                
                if img is not None:
                    resized = cv2.resize(img, self.target_size)
                    self.last_image = resized
                    return resized
            
            return self.last_image
            
        except Exception as e:
            return self.last_image
    
    def stop(self):
        self.running = False
    
    @property
    def value(self):
        return self.read()
    
    @property
    def output_size(self):
        return self.target_size
    
    def capture_image(self, filepath):
        img = self.read()
        if img is not None:
            cv2.imwrite(filepath, img)
            return True
        return False
    
    def __enter__(self):
        self.start()
        return self
    
    def __exit__(self, *args):
        self.stop()

def create_inference_camera():
    return AutonomousRacecarCamera('inference')

def create_training_camera():
    return AutonomousRacecarCamera('training')

def create_debug_camera():
    return AutonomousRacecarCamera('debug')

def test_camera(mode='debug'):
    camera = AutonomousRacecarCamera(mode)
    if camera.start():
        for i in range(3):
            img = camera.read()
            if img is not None:
                print(f"capture {i+1}: {img.shape}")
                cv2.imwrite(f"test_frame_{i}.jpg", img)
            time.sleep(0.5)
        camera.stop()
        return True
    return False

def quick_camera_test():
    return test_camera('debug')

if __name__ == "__main__":
    test_camera()

capture 1: (480, 640, 3)
capture 2: (480, 640, 3)
capture 3: (480, 640, 3)


# IMPORTANT NOTE - After a lot of testing, I figured that the res to use my cam with was 640 x 480. I am using a yahboom imx 219 cam with a 160 degree lens